In [1]:
# Import necessary libraries
import ephem
import math


# get_passes() function definition
def get_passes(observer, tle, start_time, num_passes=None, duration=None):
    """Config obs and sat, Return pass data for all passes in given interval.

    Arguments:
    observer -- 4 element list containing desired [name,lat,lon,alt]
    tle -- 3 element list containing desired tle [line0,line1,line2]
    start_time -- ephem.date string formatted 'yyyy/mm/dd hr:min:sec'
    num_passes -- integer number of desired passes (defualt None)
    duration -- float number of hours or fraction of hours (default None)

    Specify either num_passes or duration.
    If both, use num_passes.
    If neither, find passes for next 24 hours.
    """

    obs_name, obs_lat, obs_lon, obs_alt = observer
    tle_line0, tle_line1, tle_line2 = tle

    # Set up location of observer
    ground_station = ephem.Observer()
    ground_station.name = obs_name                # name string
    ground_station.lon = obs_lon                  # in degrees (+E)
    ground_station.lat = obs_lat                  # in degrees (+N)
    ground_station.elevation = obs_alt            # in meters
    ground_station.date = ephem.date(start_time)  # in UTC

    # Read in most recent satellite TLE data
    sat = ephem.readtle(tle_line0, tle_line1, tle_line2)

    contacts = []

    if num_passes is not None and duration is None:
        # if only num_passes specified
        try:
            for i in range(num_passes):
                sat.compute(ground_station)  # compute all body attributes for sat
                # next pass command yields array with [0]=rise time,
                # [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                # [4]=set time, [5]=set azimuth
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = (set_time-rise_time)  # fraction of a day

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time  # new obs time = prev set time

                pass_data = {
                    'start' : rise_time,
                    'end' : set_time,
                    'duration' : pass_duration,
                    'rise_az' : (rise_az*deg_per_rad),
                    'set_az' : (set_az*deg_per_rad)
                }

                # increase by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute
                contacts.append(pass_data)
        except ValueError:
            # No (more) visible passes
            pass
        return contacts

    if num_passes is None and duration is not None:
        # if only duration specified
        try:
            end_time = ephem.date(ground_station.date+duration*ephem.hour)
            while (ground_station.date <= end_time):
                sat.compute(ground_station)  # compute all body attributes for sat
                # next pass command yields array with [0]=rise time,
                # [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                # [4]=set time, [5]=set azimuth
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = (set_time-rise_time)  # fraction of a day

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time  # new obs time = prev set time

                pass_data = {
                    'start' : rise_time,
                    'end' : set_time,
                    'duration' : pass_duration,
                    'rise_az' : (rise_az*deg_per_rad),
                    'set_az' : (set_az*deg_per_rad)
                }

                # increase time by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute
        except ValueError:
            # No (more) visible passes
            pass
        return pass_data

    if num_passes is not None and duration is not None:
        # if both are specified, use num_passes
        try:
            for i in range(num_passes):
                sat.compute(ground_station)  # compute all body attributes for sat
                # next pass command yields array with [0]=rise time,
                # [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                # [4]=set time, [5]=set azimuth
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = (set_time-rise_time)  # fraction of a day

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time   # new obs time = prev set time

                pass_data = {
                    'start' : rise_time,
                    'end' : set_time,
                    'duration' : pass_duration,
                    'rise_az' : (rise_az*deg_per_rad),
                    'set_az' : (set_az*deg_per_rad)
                }

                # increase time by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute
        except ValueError:
            # No (more) visible passes
            pass
        return pass_data

    if num_passes is None and duration is None:
        # if neither are specified, get passes for the next 24 hours
        try:
            end_time = ephem.date(ground_station.date+1)
            while (ground_station.date <= end_time):
                sat.compute(ground_station)  # compute all body attributes for sat
                # next pass command yields array with [0]=rise time,
                # [1]=rise azimuth, [2]=max alt time, [3]=max alt,
                # [4]=set time, [5]=set azimuth
                info = ground_station.next_pass(sat)
                rise_time, rise_az, max_alt_time, max_alt, set_time, set_az = info
                deg_per_rad = 180.0/math.pi           # use to conv azimuth to deg
                pass_duration = (set_time-rise_time)  # fraction of a day

                if set_time > rise_time:  # only update if set time > rise time
                    ground_station.date = set_time   # new obs time = prev set time

                pass_data = {
                    'start' : rise_time,
                    'end' : set_time,
                    'duration' : pass_duration,
                    'rise_az' : (rise_az*deg_per_rad),
                    'set_az' : (set_az*deg_per_rad)
                }

                # increase time by 1 min and look for next pass
                ground_station.date = ground_station.date + ephem.minute
        except ValueError:
            # No (more) visible passes
            pass
        return pass_data


In [2]:
import requests

In [8]:
# itertools has lots of things that deal with looping
# "generators" and "iterators" are really nice concepts that
#can be used in place of most usage of using indexes in a loop
#because usually we aren't actually interested in counting the lines
from itertools import islice
data = []
with open('amateur.txt') as f:
    #we don't know we've reached the end of the file
    #until we actually reached it
    while True:
        #an iterator that returns the next N lines and stops
        tripleline = islice(f, 3)
        #loop over these N lines, removing trailing spaces and \n
        # this is called a "list comprehension"
        # it is pretty much exactly the same as
        #    tle = []
        #    for line in tripleline:
        #      tle.append(x.rstrip())
        #
        tle = [x.rstrip() for x in tripleline]
        
        #only accept complete data
        #the end of the file *should* have len(tle)==0 but
        #this also handles extra junk at the end
        if len(tle) == 3:
            data.append(tle)
        else:
            break

# now, data is a list of 3-element lists of strings

In [11]:
# get NORAD ID for each sat
print(data[0][2][2:7])
print(data[1][2][2:7])

07530
14781


In [31]:
vu = ['Valparaiso University', '41.4639', '-87.0439', 245.089]
start_time = '2017/6/8 12:00:00'
num_passes = 10
duration = None
vu_passes = {}
i = 0

for tle in data:
    # use NORAD ID as key for each satellite
    # value is list of passes, where each pass is a dictionary of data
    vu_passes[data[i][2][2:7]] = get_passes(vu, tle, start_time, num_passes=num_passes, duration=duration)
    i = i + 1

print(vu_passes['25544'])

[{'rise_az': 190.456669458409, 'end': 42893.31604054287, 'duration': 0.006596710234589409, 'set_az': 70.48850846101539, 'start': 42893.30944383264}, {'rise_az': 238.95741928294103, 'end': 42893.383323068636, 'duration': 0.007673796222661622, 'set_az': 53.80764288509183, 'start': 42893.37564927241}, {'rise_az': 277.83698705767466, 'end': 42893.4503480843, 'duration': 0.007075859924952965, 'set_az': 52.31765419312647, 'start': 42893.44327222437}, {'rise_az': 302.7234369967764, 'end': 42893.517890297146, 'duration': 0.006818195070081856, 'set_az': 68.37534355446182, 'start': 42893.511072102076}, {'rise_az': 308.7222555410152, 'end': 42893.58574590428, 'duration': 0.0074637355719460174, 'set_az': 101.9204174741996, 'start': 42893.57828216871}, {'rise_az': 299.42355404268983, 'end': 42893.65275732826, 'duration': 0.007447332063748036, 'set_az': 145.65946676791137, 'start': 42893.645309996195}, {'rise_az': 266.29276524705915, 'end': 42893.717289505425, 'duration': 0.003657167195342481, 'set_

In [32]:
import pprint
pprint.pprint(vu_passes)

{'07530': [{'duration': 0.008818315858661663,
            'end': 42893.05479798677,
            'rise_az': 3.1217117066147058,
            'set_az': 291.59949038617015,
            'start': 42893.04597967091},
           {'duration': 0.006794338252802845,
            'end': 42893.20346479669,
            'rise_az': 53.02774114980851,
            'set_az': 358.85267484113393,
            'start': 42893.196670458434},
           {'duration': 0.013263459732115734,
            'end': 42893.28268661655,
            'rise_az': 110.45268295519035,
            'set_az': 351.6644744755858,
            'start': 42893.26942315682},
           {'duration': 0.01566258354432648,
            'end': 42893.361458622756,
            'rise_az': 158.72548204636234,
            'set_az': 343.69645775211893,
            'start': 42893.34579603921},
           {'duration': 0.01335760948859388,
            'end': 42893.439292180774,
            'rise_az': 209.18142816232387,
            'set_az': 329.73866565

: 42893.582015750464},
           {'duration': 0.007548331268480979,
            'end': 42893.66235052971,
            'rise_az': 343.1746859410433,
            'set_az': 70.48610423442754,
            'start': 42893.65480219844},
           {'duration': 0.010617993371852208,
            'end': 42893.73697198859,
            'rise_az': 336.7238727982898,
            'set_az': 122.86446173380251,
            'start': 42893.72635399522},
           {'duration': 0.010732991009717807,
            'end': 42893.80889921308,
            'rise_az': 323.6028061951285,
            'set_az': 172.54978492646816,
            'start': 42893.79816622207},
           {'duration': 0.004635477205738425,
            'end': 42893.876679278925,
            'rise_az': 290.0851554849083,
            'set_az': 239.00498472032098,
            'start': 42893.87204380172},
           {'duration': 0.01013549230992794,
            'end': 42894.34921623677,
            'rise_az': 174.44378272293437,
            'se

'start': 42893.27623944108},
           {'duration': 0.014735654476680793,
            'end': 42893.67107842641,
            'rise_az': 147.348149077927,
            'set_az': 17.35157478449318,
            'start': 42893.65634277193},
           {'duration': 0.01640518351632636,
            'end': 42893.75259257059,
            'rise_az': 197.57733291344408,
            'set_az': 6.690843492547559,
            'start': 42893.736187387076},
           {'duration': 0.012645725029869936,
            'end': 42893.83326271642,
            'rise_az': 253.19027675844032,
            'set_az': 357.04704603214935,
            'start': 42893.82061699139},
           {'duration': 0.013078772637527436,
            'end': 42894.092042737866,
            'rise_az': 2.3288050777082687,
            'set_az': 110.2630086019391,
            'start': 42894.07896396523},
           {'duration': 0.016530722990864888,
            'end': 42894.17621297081,
            'rise_az': 352.86270822205944,
        

131.42212868831655,
            'start': 42894.70205656657},
           {'duration': 0.008103684704110492,
            'end': 42894.78504255811,
            'rise_az': 226.7482785566248,
            'set_az': 140.0454610815111,
            'start': 42894.77693887341},
           {'duration': 0.006477335162344389,
            'end': 42894.859271028174,
            'rise_az': 227.11046982793223,
            'set_az': 160.17779882996052,
            'start': 42894.85279369301},
           {'duration': 0.005544359162740875,
            'end': 42895.62461757798,
            'rise_az': 192.64649640820681,
            'set_az': 135.66370348365115,
            'start': 42895.61907321882},
           {'duration': 0.007897087409219239,
            'end': 42895.700827194945,
            'rise_az': 216.3499712434407,
            'set_az': 131.95924110426802,
            'start': 42895.692930107536}],
 '37855': [{'duration': 0.007856154974433593,
            'end': 42893.053536933,
            'ris

31.620405158803635,
            'set_az': 148.14771834292534,
            'start': 42893.018512748524},
           {'duration': 0.009842003688390832,
            'end': 42893.094967294426,
            'rise_az': 8.100109166920296,
            'set_az': 203.27789075700676,
            'start': 42893.08512529074},
           {'duration': 0.006571139820152894,
            'end': 42893.15974162856,
            'rise_az': 345.0949799658141,
            'set_az': 260.4738538855585,
            'start': 42893.15317048874},
           {'duration': 0.007378551359579433,
            'end': 42893.48369057068,
            'rise_az': 113.9760906800548,
            'set_az': 8.591570306006687,
            'start': 42893.476312019324},
           {'duration': 0.009323871869128197,
            'end': 42893.55135028173,
            'rise_az': 170.99953689055036,
            'set_az': 346.3669251138105,
            'start': 42893.54202640986},
           {'duration': 0.005927997590333689,
            'e

345.32608624657064,
            'start': 42894.473389941355},
           {'duration': 0.004485021090658847,
            'end': 42894.5456648453,
            'rise_az': 242.29437650384244,
            'set_az': 307.9653886187078,
            'start': 42894.541179824206}],
 '40903': [{'duration': 0.007808982772985473,
            'end': 42893.06592228829,
            'rise_az': 10.761988419504238,
            'set_az': 195.15866730569417,
            'start': 42893.05811330552},
           {'duration': 0.004981075813702773,
            'end': 42893.12842424862,
            'rise_az': 339.41253578393025,
            'set_az': 259.10246118324665,
            'start': 42893.12344317281},
           {'duration': 0.007411040634906385,
            'end': 42893.50388238279,
            'rise_az': 144.92882777201677,
            'set_az': 358.52299527027577,
            'start': 42893.49647134216},
           {'duration': 0.006764592544641346,
            'end': 42893.56801851167,
            'r

,
           {'duration': 0.006289876684604678,
            'end': 42894.05554971941,
            'rise_az': 34.90337997959987,
            'set_az': 147.46195368987773,
            'start': 42894.04925984272},
           {'duration': 0.0074119282071478665,
            'end': 42894.12053530714,
            'rise_az': 3.3815331698590483,
            'set_az': 209.96632618098505,
            'start': 42894.113123378935},
           {'duration': 0.0020298062736401334,
            'end': 42894.18144343601,
            'rise_az': 319.3480081535607,
            'set_az': 287.89367562933137,
            'start': 42894.17941362974},
           {'duration': 0.003980230569140986,
            'end': 42894.49246772064,
            'rise_az': 90.75169214339475,
            'set_az': 27.602524181428834,
            'start': 42894.488487490074},
           {'duration': 0.007658539267140441,
            'end': 42894.55802780857,
            'rise_az': 158.62544709577836,
            'set_az': 352.5541

22.381143381730272,
            'set_az': 170.41234552760437,
            'start': 42893.128236425735},
           {'duration': 0.008913441386539489,
            'end': 42893.20479664354,
            'rise_az': 0.8850034055710078,
            'set_az': 224.74010731799268,
            'start': 42893.195883202156},
           {'duration': 0.001928180099639576,
            'end': 42893.26779889947,
            'rise_az': 323.5684913247383,
            'set_az': 299.84932071479176,
            'start': 42893.26587071937},
           {'duration': 0.004284303206077311,
            'end': 42893.52875665996,
            'rise_az': 77.67440022262407,
            'set_az': 24.78622886586289,
            'start': 42893.52447235675},
           {'duration': 0.009421848677447997,
            'end': 42893.59793489197,
            'rise_az': 142.16542885647797,
            'set_az': 356.47374727397516,
            'start': 42893.588513043294},
           {'duration': 0.009231802578142378,
           